In [1]:
# import packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
from imutils.video import VideoStream
from imutils import face_utils
import cv2

from threading import Thread
import numpy as np
import time
import dlib



In [2]:
HAARCASCADE = "haarcascade_frontalface_default.xml"
MODEL = "model.h5"

In [ ]:
# load the face detector cascade and smile detector CNN
detector = cv2.CascadeClassifier(HAARCASCADE)
dlib_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
model = load_model(MODEL)

camera = cv2.VideoCapture(0)

# keep looping
while True:
    # grab the current frame
    (grabbed, frame) = camera.read()

    # if we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
#     if args.get("video") and not grabbed:
#         break

    # reszie the frame, convert it to grayscale, then clone the
    # original frame so we can draw on it later in the program
    if not grabbed:
        continue
    frame = imutils.resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces in the input frame, then clone the frame so
    # that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
        minSize = (30, 30), flags = cv2.CASCADE_SCALE_IMAGE)

    label_written = False

    for (fX, fY, fW, fH) in rects:
        # extract the ROI of the face from the grayscale image,
        # resize it to a fixed 28x28 pixels, and then prepare the
        # ROI for classification via CNN
        roi = gray[fY: fY + fH, fX: fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)

        # determine the probabilities of both "smiling" and "not similing"
        # then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "E" if smiling > notSmiling else "Others"
        # print(label)

        if not label_written:
            cv2.putText(frame, label + " detected", (10, 30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            label_written = True

    rects = dlib_detector(gray, 0)
    for rect in rects:
		# determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

		# draw lip contours
        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

    # show our detected faces along with smiling/not smiling labels
    cv2.imshow("Face", frame)

    # if 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()


2022-04-30 11:08:56.299329: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
